
# Two-photon Functional Preprocessing (L395_f10)

This notebook loads repository metadata, derives the preprocessing parameters for a two-photon functional session, and runs the preprocessing pipeline. Output artefacts mirror the existing `02_reg/00_preprocessing/2p_functional` layout, but everything is written into `D:/pipelineTestOutput` for testing.

> Set `REPROCESS = True` if you need to overwrite existing outputs.


In [1]:
from pathlib import Path
import logging

from social_imaging_scripts.metadata.loader import load_animals
from social_imaging_scripts.preprocessing.two_photon import functional, motion

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

In [2]:

ANIMAL_ID = "L395_f10"
OUTPUT_BASE = Path("D:/pipelineTestOutput")
REPROCESS = False
METADATA_BASE = Path(r"D:/Documents/social_imaging_scripts/metadata/animals")


In [ ]:
MOTION_REPROCESS = False
FPS = 2.0  # Hz
FAST_DISK = None  # Path('D:/pipelineTestOutput/temp_suite2p') if you want binaries elsewhere
OPS_PATH = Path('D:/pipelineTest/ressources/suite2p_ops_may2025.npy')

ops_template = motion.load_global_ops(OPS_PATH)
ops_template['nonrigid'] = True

In [5]:

collection = load_animals(base_dir=METADATA_BASE)
animal = collection.by_id(ANIMAL_ID)

functional_sessions = [s for s in animal.sessions if s.session_type == "functional_stack"]
if not functional_sessions:
    raise RuntimeError(f"No functional sessions found for {ANIMAL_ID}")

session = functional_sessions[0]
settings = session.session_data.preprocessing_two_photon
if settings is None:
    raise RuntimeError("Two-photon preprocessing settings missing in metadata")

raw_dir = Path(session.session_data.raw_path)
output_root = OUTPUT_BASE / ANIMAL_ID / "02_reg" / "00_preprocessing" / "2p_functional"

print("Raw directory:", raw_dir)
print("Output root:", output_root)
print("Mode:", settings.mode)
print("Planes:", settings.n_planes)
print("Frames per plane:", settings.frames_per_plane)
print("Blocks:", settings.blocks)


Raw directory: D:\pipelineTest\L395_f10\01_raw\2p\functional
Output root: D:\pipelineTestOutput\L395_f10\02_reg\00_preprocessing\2p_functional
Mode: resonant
Planes: 5
Frames per plane: 3
Blocks: [2, 3]


In [12]:

def preprocess_functional_session(*, animal, session, settings, output_root, reprocess=False):
    output_root = Path(output_root)
    metadata_path = output_root / "01_individualPlanes" / f"{animal.animal_id}_preprocessing_metadata.json"

    if metadata_path.exists() and not reprocess:
        print(f"Skipping preprocessing for {animal.animal_id} (metadata exists at {metadata_path})")
        return {"metadata": metadata_path}

    results = functional.run(
        animal_id=animal.animal_id,
        session_id=session.session_id,
        raw_dir=session.session_data.raw_path,
        output_root=output_root,
        settings=settings,
    )
    print("Generated artefacts:")
    for key, path in results.items():
        print(f"  {key}: {path}")
    return results


In [13]:

results = preprocess_functional_session(
    animal=animal,
    session=session,
    settings=settings,
    output_root=output_root,
    reprocess=REPROCESS,
)


Skipping preprocessing for L395_f10 (metadata exists at D:\pipelineTestOutput\L395_f10\02_reg\00_preprocessing\2p_functional\01_individualPlanes\L395_f10_preprocessing_metadata.json)


In [14]:
plane_dir = output_root / '01_individualPlanes'
plane_tiffs = sorted(plane_dir.glob(f'{ANIMAL_ID}_plane*.tif'))
if not plane_tiffs:
    raise FileNotFoundError(f'No plane TIFFs found in {plane_dir}')

motion_results = []
for plane_path in plane_tiffs:
    try:
        plane_idx = int(plane_path.stem.split('plane')[-1])
    except ValueError as err:
        raise ValueError(f'Unexpected plane filename: {plane_path.name}') from err

    res = motion.run_motion_correction(
        animal=animal,
        plane_idx=plane_idx,
        plane_tiff=plane_path,
        ops_template=ops_template,
        fps=FPS,
        output_root=output_root,
        fast_disk=FAST_DISK,
        reprocess=MOTION_REPROCESS,
    )
    motion_results.append(res)
    print(f'Processed plane {plane_idx}')

{}
FOUND BINARIES AND OPS IN ['D:\\pipelineTestOutput\\L395_f10\\02_reg\\00_preprocessing\\2p_functional\\suite2p\\plane0\\ops.npy']
removing previous detection and extraction files, if present
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
NOTE: Applying builtin classifier at C:\Users\jlarsch\AppData\Local\miniconda3\envs\antspy-win\Lib\site-packages\suite2p\classifiers\classifier.npy
----------- REGISTRATION
Reference frame, 5.27 sec.


MemoryError: Unable to allocate 6.09 GiB for an array with shape (3120, 512, 512) and data type complex64

In [ ]:
print('Motion correction outputs:')
for res in motion_results:
    for key, path in res.items():
        print(f'  {key}: {path}')

In [ ]:

from itertools import islice

plane_dir = output_root / "01_individualPlanes"
if plane_dir.exists():
    print("Sample outputs:")
    for path in islice(sorted(plane_dir.glob("*.tif")), 5):
        print("  ", path)
    metadata_file = plane_dir / f"{animal.animal_id}_preprocessing_metadata.json"
    if metadata_file.exists():
        print("Metadata:", metadata_file)
